In [44]:
import pandas as pd
import pathlib

In [148]:
#maximal e value for hmmsearch can be modified through e value cutoff argument
def combine_annotations(worm, e_value_cutoff=0.001):
    # read eggnogg annotation file
    eggnogg_file = f'C:/worms/new/eggnog_mapper/{worm}/{worm}.emapper.annotations.xlsx'
    eggnog_annotation_df = pd.read_excel(eggnogg_file)
    print(f'Number of proteins annotated in eggnog {eggnog_annotation_df.shape[0]}')
    hmmsearch_file = f'C:/worms/new/hmmer_results/{worm}/seq.txt'
    contig_domains = {}
    #save data about domains found during hmmsearch for every contig
    with open (hmmsearch_file, 'r') as hmmsearch_data:
        for hmm_line in hmmsearch_data:
            if hmm_line.startswith('#'):
                continue
            hmm_line_list = hmm_line.split()
            if float(hmm_line_list[4]) > e_value_cutoff:
                continue
            if contig_domains.get(hmm_line_list[0]) == None:
                contig_domains[hmm_line_list[0]] = [hmm_line_list[2]]
            else:
                contig_domains[hmm_line_list[0]].append(hmm_line_list[2])
    contig_domains = {contig:', '.join(domains) for contig, domains in contig_domains.items()}
    print(f'Number of proteins with at least one hmmer annotation {len(contig_domains)}, e-value < {e_value_cutoff}')
    eggnog_annotation_df = eggnog_annotation_df.set_index('query')
    #combine eggnog and hmmer data
    eggnog_annotation_df['hmmsearch_pfam_domains'] = pd.Series(contig_domains)
    contigs_with_egg_annot = set(eggnog_annotation_df.index)
    contigs_with_hmmer_hits = set(contig_domains.keys())
    print(f'Number of proteins annotated only in eggnog {len(contigs_with_egg_annot.difference(contigs_with_hmmer_hits))}')
    print(f'Number of proteins annotated only in hmmsearch {len(contigs_with_hmmer_hits.difference(contigs_with_egg_annot))}')
    print(f'Number of proteins with double annotation {len(contigs_with_egg_annot.intersection(contigs_with_hmmer_hits))}')
    
    #add additional contigs that were not annotated in eggnogg but have significant domain hits in hmmsearch 
    eggnog_annotation_df.reset_index(inplace=True)
    contigs_to_add = list(contigs_with_hmmer_hits.difference(contigs_with_egg_annot))
    additional_contigs_domains = dict(filter(lambda item: item[0] in contigs_to_add, contig_domains.items()))
    new_rows = {'query': [],
         'seed_ortholog': ['-']*len(additional_contigs_domains),
         'evalue': ['-']*len(additional_contigs_domains),
         'score': ['-']*len(additional_contigs_domains),
         'eggNOG_OGs': ['-']*len(additional_contigs_domains),
         'max_annot_lvl': ['-']*len(additional_contigs_domains),
         'COG_category': ['-']*len(additional_contigs_domains),
         'Description': ['-']*len(additional_contigs_domains),
         'Preferred_name': ['-']*len(additional_contigs_domains),
         'GOs': ['-']*len(additional_contigs_domains),
         'EC': ['-']*len(additional_contigs_domains),
         'KEGG_ko': ['-']*len(additional_contigs_domains),
         'KEGG_Pathway': ['-']*len(additional_contigs_domains),
         'KEGG_Module': ['-']*len(additional_contigs_domains),
         'KEGG_Reaction': ['-']*len(additional_contigs_domains),
         'KEGG_rclass': ['-']*len(additional_contigs_domains),
         'BRITE': ['-']*len(additional_contigs_domains),
         'KEGG_TC': ['-']*len(additional_contigs_domains),
         'CAZy': ['-']*len(additional_contigs_domains),
         'BiGG_Reaction': ['-']*len(additional_contigs_domains),
         'PFAMs': ['-']*len(additional_contigs_domains),
         'hmmsearch_pfam_domains': []}
    for contig, domains in additional_contigs_domains.items():
        new_rows['query'].append(contig)
        new_rows['hmmsearch_pfam_domains'].append(domains)
    #save combined annotation file
    pd.concat([eggnog_annotation_df, pd.DataFrame.from_dict(new_rows)])\
        .to_csv(f'C:/worms/new/{worm}_combined_annotation.csv', index=False, sep='\t')

In [149]:
combine_annotations('Pdum')

Number of proteins annotated in eggnog 25008
Number of proteins with at least one hmmer annotation 33138, e-value < 0.001
Number of proteins annotated only in eggnog 1191
Number of proteins annotated only in hmmsearch 9321
Number of proteins with double annotation 23817


In [150]:
combine_annotations('Pele')

Number of proteins annotated in eggnog 23720
Number of proteins with at least one hmmer annotation 32259, e-value < 0.001
Number of proteins annotated only in eggnog 1417
Number of proteins annotated only in hmmsearch 9956
Number of proteins with double annotation 22303
